# <center>Big Data for Engineers &ndash; Exercises</center>
## <center>Spring 2025 &ndash; Week 9 &ndash; ETH Zurich</center>
## <center>Spark Dataframes and SparkSQL</center>

# Preparation for the exercise in Spark

1. Drag this notebook (and everything else in the exercise09 folder) in the `notebooks` folder of your exam magic box

2. Start docker with ```docker-compose up -d```

3. Launch Jupiter from the docker container

## 1. Spark Dataframes

Spark Dataframes allow the user to perform simple and efficient operations on data, as long as the data is structured and has a schema. Dataframes are similar to relational tables in relational databases: conceptually a dataframe is a specialization of a Spark RDD with schema information attached. You can find more information in Karau, H. et al. (2015). Learning Spark, Chapter 9 (optional reading).

Throughout the exercise, you can see the equivalency of Spark RDD, Spark Dataframes and SparkSQL. 

### 1.1. Data preprocessing

In [ ]:
import json
from pyspark.sql import SparkSession
from pyspark import SparkConf

spark = SparkSession.builder.master('local').getOrCreate()
sc = spark.sparkContext

path = "orders.jsonl"
orders_df = spark.read.json(path).cache()

The type of our dataset object is DataFrame.

In [ ]:
type(orders_df)

Print the schema.

In [ ]:
orders_df.printSchema()

Print one row.

In [ ]:
orders_df.limit(1).collect()

You can access the underlying RDD object and use any functions you learned for Spark RDDs.

In [ ]:
orders_df.rdd.filter(lambda ordr: ordr.customer.last_name == "Landry").count()

### 1.2. Dataframe Operations
We perform some queries using operations on Dataframes ([Here](https://spark.apache.org/docs/latest/sql-programming-guide.html#untyped-dataset-operations-aka-dataframe-operations) is a guide on DF Operations with a link to the [API Documentation](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/index.html))

We can select columns and show the results.

In [ ]:
orders_df.select("customer.first_name", "customer.last_name").limit(5).show()

You can also convert the dataframe object to a pandas dataframe to display it nicely in jupyter.

In [ ]:
orders_df.select("customer.first_name", "customer.last_name").limit(5).toPandas()

As you can see we can navigate to the nested items with the dot notation.

In [ ]:
orders_df.filter(orders_df["customer.last_name"] == "Landry").count()

How about nested arrays?

In [ ]:
orders_df.select("order_id", "items").orderBy("order_id").limit(5).show()

Let us try to find orders of a fan.

In [ ]:
orders_df.filter(orders_df["items.product"] == "fan").count()

The above code doesn't work! Use ```array_contains``` instead. We have to import it first. [Here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.array_contains.html) you can learn more about it.

In [ ]:
from pyspark.sql.functions import array_contains

orders_df.filter(array_contains("items.product", "fan")).count()

#### Let us try to unnest the data.

We can unnest the products with ```explode```.

[Explode](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.explode.html) will generate as many rows as there are elements in the array and match them to other attributes. You should name the newly generated exploded column in order to be able to refer to it. 

Here you can find the syntax. Note that with newer Spark versions (like the one in the exam magic box) you have to use ```select``` twice since newly exploded columns are not immediately available.

In [ ]:
from pyspark.sql.functions import explode

orders_df.select(explode("items").alias("i"), "order_id").select("i.product", "order_id").orderBy("order_id").limit(5).show()

Now we can use this table to filter.

In [ ]:
exploded_df = orders_df.select(explode("items").alias("i"), "order_id").select("i.product", "order_id")
exploded_df.filter(exploded_df["product"] == "fan").count()

You might have tried to access the ```i.product``` column directly using a ```.filter``` right after the ```.select```. That, however, does not work, because the column is not available to ```orders_df``` when creating a clause like ```(orders_df["i.product"] == "fan")```. In order to filter on a freshly exploded column is best to proceed in stps and create an intermediate table. 

A possible workaround when using Dataframe operations is that of using a string clause in ```.filter```, so that the product column will be resolved after it has been added with the ```.select```.

In [ ]:
orders_df.select(explode("items").alias("i"), "order_id").select("i.product", "order_id").filter("product = 'fan'").count()

Any ideas why there are more "fan" in the `explode` query than the `array_contain` one? 

This is because that there could be more than one "fan" types in each order, and we would therefore count some orders with the same `order_id` more than one time. You will find about that when inspecting the `orders.jsonl` data. 
E.g., 
```json
{"order_id": 2, "date": "2016-6-6", "customer": {"first_name": "Brendon", "last_name": "Sicilia"}, "items": [..., {"product": "fan", "quantity": 7, "price": 1.1}, ..., {"product": "fan", "quantity": 8, "price": 1.15}]}
```

#### Project the nested columns.

In [ ]:
orders_df.select(explode("items").alias("i"), "*").select(
    "order_id", "customer.*", "date", "i.*").limit(3).show()

Just like in SQL ```*``` is a shortcut for just selecting all of the fields involved.

### 1.3. Exercises

#### 1. Find the number of distinct products

#### 2. Find the average quantity at which each product is purchased. Only show the top 10 products by average quantity.
Hint: You may need to import the function [`desc`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.desc.html) from ```pyspark.sql.functions``` to define descending order.

#### 3. Find the most expensive order.
Hint: You can first build a dataframe with `explode`. Then you can calculate the total price and aggregate per order. 

## 2. Spark SQL

Spark SQL allows the users to formulate their queries using SQL. The requirement is the use of Dataframes, which as said before are similar to relational tables. In addition to a familiar interface, writing queries in SQL might provide better performance than RDDs, inheriting efficiency from the Dataframe operations, while also performing automatic optimization of queries.

First we need to install the `sparksql` magic command.

In [ ]:
!pip install sparksql-magic --quiet

In [ ]:
%load_ext sparksql_magic

In order to use SQL we need to create a temporary table.

Note that this table only exists for the current session.

In [ ]:
orders_df.createOrReplaceTempView("orders")

### 2.1. Queries

Finally, run SQL queries on the registered table `orders`. We will run the same queries as during the previous section, but with SQL.

As you can see we can navigate to the nested items with the dot.

In [ ]:
%%sparksql
SELECT count(*)
FROM orders
WHERE orders.customer.last_name == "Landry"

How about nested arrays?

In [ ]:
%%sparksql
SELECT order_id, items
FROM orders AS o
ORDER BY order_id
LIMIT 5

Let us try to find orders of a fan.

In [ ]:
%%sparksql 
SELECT count(*)
FROM orders
WHERE items.product = "fan"

The above code doesn't work! Use [```array_contains```](https://spark.apache.org/docs/latest/api/sql/index.html#array_contains) instead.

In [ ]:
%%sparksql
SELECT count(*)
FROM orders
WHERE array_contains(items.product, "fan")

#### Let us try to unnest the data.

Unnest the products with `explode`.

[Explode](https://spark.apache.org/docs/latest/api/sql/index.html#explode) will generate as many rows as there are elements in the array and match them to other attributes.

Here you can find the syntax. Note that with newer Spark versions (like the one in the exam magic box) you have to use select twice since newly exploded columns are not immediately available, therefore we need to use explode in a nested query.

In [ ]:
%%sparksql
SELECT i.product, order_id
FROM (
    SELECT explode(items) AS i, order_id
    FROM orders
) exploded_items
ORDER BY order_id
LIMIT 5

Now we can use this table to filter. For example we want to find out how many times does "fan" appear.

In [ ]:
%%sparksql
SELECT count(*)
FROM (
    SELECT explode(items) AS i
    FROM orders
    )
WHERE i.product = "fan"

You might have tried to access the i.product column directly in the same ```SELECT``` clause. That, however, just like before, does not work. This is because the column is not available to the ```WHERE``` clause right away. In order to access the built columns directly, we need to unnest the data and make it part of our ```FROM``` clause. ```LATERAL VIEW``` lets us do just that, matching each non-array attribute to an unnested row from the array. [Here](https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-lateral-view.html) you can find out more about that. 

In [ ]:
%%sparksql
SELECT *
FROM orders LATERAL VIEW explode(items) as flat_items
WHERE flat_items.product = "fan"
ORDER BY order_id
LIMIT 4

With this we can also project the nested columns.

In [ ]:
%%sparksql
SELECT order_id, customer.first_name, customer.last_name, date, flat_items.*
FROM orders LATERAL VIEW explode(items) item_table as flat_items
WHERE flat_items.product = "fan"
ORDER BY order_id
LIMIT 3

Having built an unnested table, we can now easily aggregate over the previously nested columns.

### 2.2. Exercises

#### 1. Find the number of distinct products

#### 2. Find the average quantity at which each product is purchased. Only show the top 10 products by quantity. 

#### 3. Find the most expensive order.

## 3. More queries

We will now explore the dataset that is going to be used in the graded exercise of this week. It will be the same language game dataset as in exercise08. If you did not download it last week, here are the instructions again.

1. Move to the `notebooks` folder in the terminal
2. Download the data: <br>
   ```wget https://f003.backblazeb2.com/file/larsyencken-eu-public/greatlanguagegame/confusion-2014-03-02.tbz2``` <br>
   __or__ <br>
   ```curl -O https://f003.backblazeb2.com/file/larsyencken-eu-public/greatlanguagegame/confusion-2014-03-02.tbz2```
3. Extract the data: <br>
   ```tar -jxvf confusion-2014-03-02.tbz2```
4. Change directory to ```confusion-2014-03-02```
5. Extract the part of the dataset that we will work with in this exercise: <br>
   ```head -n 3000000 confusion-2014-03-02.json > confusion-part.json```

### 3.1 Data processing

In [ ]:
path = "confusion-2014-03-02/confusion-part.json"
dataset = spark.read.json(path).cache()

Have a look at the data.

In [ ]:
dataset.limit(3).show()

Print the schema.

In [ ]:
dataset.printSchema()

### 3.2 Spark Dataframe queries

#### 1. Find the number of games where the guessed language and target language is Maltese.

#### 2. Return the number of distinct "target" languages.

#### 3. Return the sample IDs (i.e., the "sample" field) of the first three games where the guessed language is correct (equal to the target one) ordered by date (descending), then by language (ascending), then by country (descending). 
Hint: passing `truncate=False` to `show()` allows you to see the full output, otherwise you can simply use `collect()` instead.

#### 4. Aggregate all games by country and "guess" language, counting the number of guesses for each group and return the frequencies of the two most frequent country/language combinations.

#### 5 Sort the languages by decreasing overall percentage of correct guesses and return the first four languages. 
Hint: `withColumnRenamed()` allows you to set the names of the generated columns and remember it is possible to `join()` Dataframes.

### 3.2 Spark SQL queries
We will now go over the same queries but using Spark SQL instead

In [ ]:
dataset.createOrReplaceTempView("dataset")

Have a look at the data in the new format.

In [ ]:
%%sparksql
SELECT *
FROM dataset
LIMIT 3

#### 1. Find the number of games where the guessed language and target language is Maltese.

#### 2. Return the number of distinct "target" languages.

#### 3. Return the sample IDs (i.e., the "sample" field) of the first three games where the guessed language is correct (equal to the target one) ordered by date (descending), then by language (ascending), then by country (descending).

#### 4. Aggregate all games by country and "guess" language, counting the number of guesses for each group and return the frequencies of the two most frequent country/language combinations.

#### 5. Sort the languages by decreasing overall percentage of correct guesses and return the first four languages.